# Importing Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

# Data

In [3]:
# Data directory
DATADIR = r'C:\Users\sweta\OneDrive\Desktop\DL Projects\Human Activity Recognition\UCI_HAR_Dataset'

In [4]:
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [5]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename =fr"C:\Users\sweta\OneDrive\Desktop\DL Projects\Human Activity Recognition\UCI_HAR_Dataset\{subset}\Inertial Signals\{signal}_{subset}.txt"
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [6]:
def load_y(subset):
    """
    from 1 to 6, that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    """
    filename = fr'C:\Users\sweta\OneDrive\Desktop\DL Projects\Human Activity Recognition\UCI_HAR_Dataset\{subset}\y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [7]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [8]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

In [9]:
# Configuring a session
session_conf =tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [10]:
# Import Keras
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf))

In [11]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [12]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [14]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [15]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [16]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


# Defining the Architecture of LSTM

In [17]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                5376      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [19]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Epoch 1/30
460/460 [==============================] - 59s 17ms/step - loss: 1.3504 - accuracy: 0.4347 - val_loss: 1.0375 - val_accuracy: 0.5996
Epoch 2/30
460/460 [==============================] - 7s 14ms/step - loss: 0.8295 - accuracy: 0.6431 - val_loss: 0.7556 - val_accuracy: 0.6916
Epoch 3/30
460/460 [==============================] - 7s 14ms/step - loss: 0.6749 - accuracy: 0.7134 - val_loss: 0.7003 - val_accuracy: 0.7431
Epoch 4/30
460/460 [==============================] - 7s 14ms/step - loss: 0.5576 - accuracy: 0.7937 - val_loss: 0.6343 - val_accuracy: 0.8022
Epoch 5/30
460/460 [==============================] - 7s 14ms/step - loss: 0.3869 - accuracy: 0.8763 - val_loss: 0.4448 - val_accuracy: 0.8690
Epoch 6/30
460/460 [==============================] - 7s 14ms/step - loss: 0.3034 - accuracy: 0.9062 - val_loss: 0.5017 - val_accuracy: 0.8402
Epoch 7/30
460/460 [==============================] - 7s 14ms/step - loss: 0.2431 - accuracy: 0.9220 - val_loss: 0.4882 - val_accuracy: 0.839

In [20]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        27        0                   0   
SITTING                  0      382       103        1                   0   
STANDING                 0       78       448        0                   0   
WALKING                  0        1         0      365                  14   
WALKING_DOWNSTAIRS       1        0         0        1                 397   
WALKING_UPSTAIRS         0        0         0        0                  18   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            5  
STANDING                           6  
WALKING                          116  
WALKING_DOWNSTAIRS                21  
WALKING_UPSTAIRS                 453  


In [21]:
score = model.evaluate(X_test, Y_test)

93/93 [==============================] - 1s 6ms/step - loss: 0.9181 - accuracy: 0.8670


In [22]:
score

[0.9180893898010254, 0.8669833540916443]

# Conclusion
- With a simple 2 layer architecture we got 86.7% accuracy and a loss of 0.91
- We can further imporve the performace with Hyperparameter tuning